In [1]:
from ray.rllib.agents import ppo
from ray.rllib.agents import dqn

In [3]:
ppo.DEFAULT_CONFIG.copy()

{'num_workers': 2,
 'num_envs_per_worker': 1,
 'create_env_on_driver': False,
 'rollout_fragment_length': 200,
 'batch_mode': 'truncate_episodes',
 'gamma': 0.99,
 'lr': 5e-05,
 'train_batch_size': 4000,
 'model': {'_use_default_native_models': False,
  'fcnet_hiddens': [256, 256],
  'fcnet_activation': 'tanh',
  'conv_filters': None,
  'conv_activation': 'relu',
  'post_fcnet_hiddens': [],
  'post_fcnet_activation': 'relu',
  'free_log_std': False,
  'no_final_linear': False,
  'vf_share_layers': False,
  'use_lstm': False,
  'max_seq_len': 20,
  'lstm_cell_size': 256,
  'lstm_use_prev_action': False,
  'lstm_use_prev_reward': False,
  '_time_major': False,
  'use_attention': False,
  'attention_num_transformer_units': 1,
  'attention_dim': 64,
  'attention_num_heads': 1,
  'attention_head_dim': 32,
  'attention_memory_inference': 50,
  'attention_memory_training': 50,
  'attention_position_wise_mlp_dim': 32,
  'attention_init_gru_gate_bias': 2.0,
  'attention_use_n_prev_actions': 0,


In [3]:
# ppo.DEFAULT_CONFIG.copy()

In [1]:
from solinv.environment import InvariantEnvironment
from solinv.tyrell.interpreter import InvariantInterpreter
from solinv.tyrell import spec as S
from solinv.tyrell import dsl as D

from solinv.tyrell.dsl.utils import derive_dfs, get_hole_dfs

In [2]:
spec = S.parse_file("./dsls/example0.tyrell")
start_type = spec.get_type("Expr")
builder = D.Builder( spec )
interpreter = InvariantInterpreter()

In [3]:
config = {
    "spec": spec,
    "start_type": start_type,
    "max_step": 10,
    "contract_path": "/Users/joseph/Desktop/UCSB/21fall/SolidTypes/test/regression/good/mint_MI.sol",
    "interpreter": interpreter
}
env = InvariantEnvironment(config=config)

In [4]:
env.observation_space.sample()

OrderedDict([('action_mask',
              array([1, 1, 0, 1, 1, 0, 1, 0, 1, 1], dtype=int32)),
             ('inv', array([4], dtype=int32))])

In [7]:
env.get_curr_state()

[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1]

In [5]:
env.reset()

[1]

In [20]:
spec.productions()[1].lhs

ValueType(Expr, properties={})

In [9]:
env.action_list

[FunctionProduction(id=0, lhs=ValueType(Empty, properties={}), name=empty, rhs=[ValueType(Empty, properties={})]),
 FunctionProduction(id=1, lhs=ValueType(Expr, properties={}), name=enum2expr, rhs=[EnumType(EnumExpr, domain=['_totalSupply', '0', '_balances'])]),
 FunctionProduction(id=2, lhs=ValueType(Expr, properties={}), name=and, rhs=[ValueType(Expr, properties={}), ValueType(Expr, properties={})]),
 FunctionProduction(id=3, lhs=ValueType(Expr, properties={}), name=leq, rhs=[ValueType(Expr, properties={}), ValueType(Expr, properties={})]),
 FunctionProduction(id=4, lhs=ValueType(Expr, properties={}), name=add, rhs=[ValueType(Expr, properties={}), ValueType(Expr, properties={})]),
 FunctionProduction(id=5, lhs=ValueType(Expr, properties={}), name=flatten, rhs=[ValueType(Expr, properties={})]),
 FunctionProduction(id=6, lhs=ValueType(Expr, properties={}), name=sum, rhs=[ValueType(Expr, properties={})]),
 EnumProduction(id=7, lhs=EnumType(EnumExpr, domain=['_totalSupply', '0', '_balanc

In [21]:
env.curr_inv

HoleNode<Expr>

In [23]:
env.curr_inv

NotImplementedError: Invalid method on a hole node

In [11]:
env.step(3)

# [debug:good] curr_inv=leq(Hole<Expr>, Hole<Expr>), seq=[3], action=3


[[3, -1, -1, -1, -1, -1, -1, -1, -1, -1], 0.0, False, {'info': 'good'}]

In [12]:
env.step(1)

# [debug:good] curr_inv=leq(enum2expr(Hole<EnumExpr>), Hole<Expr>), seq=[3, 1], action=1


[[3, 1, -1, -1, -1, -1, -1, -1, -1, -1], 0.0, False, {'info': 'good'}]

In [13]:
env.step(7)

# [debug:good] curr_inv=leq(enum2expr(_totalSupply), Hole<Expr>), seq=[3, 1, 7], action=7


[[3, 1, 7, -1, -1, -1, -1, -1, -1, -1], 0.0, False, {'info': 'good'}]

In [14]:
env.step(1)

# [debug:good] curr_inv=leq(enum2expr(_totalSupply), enum2expr(Hole<EnumExpr>)), seq=[3, 1, 7, 1], action=1


[[3, 1, 7, 1, -1, -1, -1, -1, -1, -1], 0.0, False, {'info': 'good'}]

In [15]:
env.step(7)

# [debug:good] curr_inv=leq(enum2expr(_totalSupply), enum2expr(_totalSupply)), seq=[3, 1, 7, 1, 7], action=7
# [debug] seq:[3, 1, 7, 1, 7] checking (_totalSupply <= _totalSupply)
# [debug] checking good!!!!!!!!!!!!!!!!!!!!!!


[[3, 1, 7, 1, 7, -1, -1, -1, -1, -1], 100.0, True, {'info': 'good'}]

In [15]:
env.curr_inv

ApplyNode(leq, [ApplyNode(enum2expr, [AtomNode(_totalSupply)]), HoleNode<Expr>])

In [16]:
derive_dfs(builder, env.curr_inv, env.action_list[1])

# [debug] node=leq(enum2expr(_totalSupply), Hole<Expr>)
# [debug] i=0
# [debug] node=enum2expr(_totalSupply)
# [debug] i=0
# [debug] node.production=Production 1: Expr -> enum2expr(EnumExpr), node=enum2expr(_totalSupply), tmp_clist=[]


ValueError: Argument count mismatch: expected 1 but found 0

In [1]:
import torch

In [2]:
a = torch.rand(3,4,5)

In [4]:
torch.cat([a,a],dim=1).size()

torch.Size([3, 8, 5])